In [148]:
import time
import numpy as np
import pandas as pd

from numpy.linalg import norm

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score

from sklearn.manifold import TSNE 
from sklearn.decomposition import PCA

## Загрузка датасета:

In [149]:
df = datasets.load_digits()
X = df.data
y = df.target

## Создание необходимых моделей и преобразователей:

In [150]:
models = {
    'SVC': SVC(random_state=random_state),
    'DecisionTreeClassifier': DecisionTreeClassifier(random_state=random_state),
    'LogisticRegression' : LogisticRegression(max_iter=10000, random_state=random_state),
}

In [151]:
transformers = {
    'PCA': PCA(n_components=0.90, random_state=random_state),
    't-SNE': TSNE(n_components=2, random_state=random_state)
}

### Константы

In [152]:
RANDOM_STATE = 42
TEST_SIZE = 0.2

## Обучение модели и получение предиктов (без уменьшения размерности):

In [153]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y)

predicts = {}
for model_name, model in models.items():
    model.fit(X_train, y_train)
    predicts[model_name] = model.predict(X_test)

## Обучение модели и получение предиктов (с уменьшением размерности):

In [154]:
processing_time = {}
transformed_predicts = {}

In [155]:
# Соответственно для прогона fit и predict на разных моделях из models
def get_fit_predict(X_train, X_test, transformed_predicts: dict, trans_name: str) -> dict:
    for model_name, model in models.items():
        model.fit(X_train, y_train)
        transformed_predicts[trans_name][model_name] = model.predict(X_test)
    return transformed_predicts

### PCA

In [156]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y)

trans_name = 'PCA'

transformed_predicts[trans_name] = {}

start_time = time.time()
transformers[trans_name].fit(X_train)
X_train_transformed = transformers[trans_name].transform(X_train)
X_test_transformed = transformers[trans_name].transform(X_test)
processing_time[trans_name] = time.time() - start_time

transformed_predicts = get_fit_predict(X_train_transformed, X_test_transformed, transformed_predicts, trans_name)

In [157]:
print(f'\nЧтобы описать 90% дисперсии в PCA необходимо компонентов: {transformers["PCA"].n_components_} шт.\n')


Чтобы описать 90% дисперсии в PCA необходимо компонентов: 21 шт.



### t-SNE

In [158]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y)

trans_name = 't-SNE'

transformed_predicts[trans_name] = {}

start_time = time.time()
X = transformers[trans_name].fit_transform(X)
processing_time[trans_name] = time.time() - start_time

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y)

transformed_predicts = get_fit_predict(X_train, X_test, transformed_predicts, trans_name)

## Score результаты:

In [159]:
# Для вывода разницы score
full_data_score = {}
sliced_data_score['PCA'] = {}
sliced_data_score['t-SNE'] = {}

### Без снижения размерности:

In [160]:
for model_name, predict in predicts.items():
    print(f"{model_name} score: ", round(accuracy_score(y_true=y_test, y_pred=predict), 3))
    full_data_score[model_name] = accuracy_score(y_true=y_test, y_pred=predict)

SVC score:  0.992
DecisionTreeClassifier score:  0.825
LogisticRegression score:  0.961


### Со снижением размерности:

In [161]:
for transformer_name, predicts in transformed_predicts.items():
    print(f"\n{transformer_name}\n")
    for model_name, predict in predicts.items():
        print(f"{model_name} score: ", round(accuracy_score(y_true=y_test, y_pred=predict), 3))
        sliced_data_score[transformer_name][model_name] = accuracy_score(y_true=y_test, y_pred=predict)


PCA

SVC score:  0.989
DecisionTreeClassifier score:  0.847
LogisticRegression score:  0.933

t-SNE

SVC score:  0.964
DecisionTreeClassifier score:  0.975
LogisticRegression score:  0.908


### Разница score

In [162]:
for transformer_name, predicts in transformed_predicts.items():
    print(f"\n{transformer_name}\n")
    for model_name, predict in predicts.items():
        diff = sliced_data_score[transformer_name][model_name] - full_data_score[model_name]
        print(f"{model_name}: {round(diff, 3)}")


PCA

SVC: -0.003
DecisionTreeClassifier: 0.022
LogisticRegression: -0.028

t-SNE

SVC: -0.028
DecisionTreeClassifier: 0.15
LogisticRegression: -0.053


### Время, требуемое для препроцессинга PCA и t-SNE

In [163]:
for transformer_name, time in processing_time.items():
    print(f"\n{transformer_name}: {round(time, 2)} секунд")


PCA: 0.02 секунд

t-SNE: 6.77 секунд


## Итоги:
#### PCA для сохранения 90% дисперии необходимо компонентов: 
##### - 21 шт.
#### Лучшие метрики точности моделей классификации:
**PCA**
- Для **SVC** результаты стали **хуже** на 0.003
- Для **DecisionTreeClassifier** результаты стали **лучше** на 0.022
- Для **LogisticRegression** результаты также стали **хуже** на 0.028

**t-SNE**
- Для **SVC** результаты стали **хуже** на 0.028
- Для **DecisionTreeClassifier** результаты стали **лучше** на 0.15
- Для **LogisticRegression** результаты также стали **хуже** на 0.053

То есть, улучшение результатов есть только у модели **DecisionTreeClassifier** и лучше себя показывает алгоритм **t-SNE**
#### Время работы алгоритмов уменьшения размерности:
- **PCA**: 0.02 сек.
- **t-SNE**: 6.77 сек.